In [101]:
import mlflow
from mlflow.models import infer_signature
from mlflow.models import validate_serving_input

In [5]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_score,f1_score,recall_score,RocCurveDisplay


In [7]:
dataset = pd.read_csv("heart-disease.csv")
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
np.random.seed(42)

In [ ]:
params = {"penalty":"l2","solver":"lbfgs","max_iter":1000,"multi_class":"ovr","random_state":8888,"n_jobs":200}

In [ ]:
lr = LogisticRegression(**params)
lr.fit(X_train,y_train)

InvalidParameterError: The 'max_depth' parameter of RandomForestClassifier must be an int in the range [1, inf) or None. Got [None, 3, 5, 10] instead.

In [94]:
y_pred = lr.predict(X_test)

In [95]:
y_pred

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0])

In [96]:
y_test

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0])

In [97]:
accuracy = accuracy_score(y_test,y_pred)

In [98]:
accuracy

0.8852459016393442

In [99]:
mlflow.set_experiment("MLFLOW Starter")
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric('accuracy',accuracy)
    mlflow.set_tag("Heart Disease Prediction","Heart Disease Prediction")
    signature = infer_signature(X_train,lr.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="heart-disease-prediciton",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart"
    )


2025/08/11 23:24:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 23:24:28 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/08/11 23:24:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2


🏃 View run gentle-eel-522 at: http://127.0.0.1:5000/#/experiments/428385134833405134/runs/7d58438f762642c59d794e9aced38a1e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/428385134833405134


Created version '2' of model 'tracking-quickstart'.


In [100]:
model_info.model_uri

'models:/m-80547c65eabd4db29a90f825b8651bf3'

# inferencing

In [105]:
model_uri = model_info.model_uri

serving_payload = """
{
"inputs":[
    [4.91, 3.28, 0.14, 1.25, 4.88, 2.12, 3.99, 0.85, 1.53, 4.23, 3.46, 0.18, 2.75],
    [0.27, 4.50, 1.92, 3.64, 0.45, 2.91, 4.71, 3.11, 2.05, 1.14, 0.98, 4.22, 3.38],
    [3.19, 2.37, 0.69, 4.75, 3.33, 1.25, 0.47, 4.98, 2.59, 3.17, 4.42, 0.58, 1.92],
    [4.27, 0.55, 2.44, 3.93, 0.14, 4.56, 1.78, 2.44, 0.65, 4.29, 3.84, 1.23, 0.77],
    [2.74, 1.96, 4.12, 0.84, 3.26, 4.61, 1.97, 3.84, 4.48, 2.03, 0.91, 4.55, 1.15],
    [3.65, 2.25, 4.78, 1.41, 4.25, 0.66, 3.91, 2.56, 0.34, 1.62, 4.12, 3.25, 2.44],
    [1.33, 4.73, 0.86, 4.26, 0.13, 1.58, 4.55, 2.94, 3.76, 1.92, 2.61, 4.89, 0.73],
    [0.95, 3.49, 4.28, 2.81, 1.53, 0.94, 4.63, 3.12, 2.59, 4.85, 0.16, 3.27, 1.65],
    [4.58, 0.83, 3.62, 1.95, 4.29, 0.19, 2.03, 1.56, 4.72, 3.88, 0.48, 1.77, 2.21],
    [1.74, 4.02, 2.95, 3.64, 0.56, 4.53, 2.75, 0.83, 3.14, 4.69, 1.97, 2.08, 4.28],
    [3.45, 0.78, 4.66, 0.17, 4.95, 1.84, 3.53, 2.41, 1.38, 4.76, 2.57, 0.94, 3.11],
    [2.58, 3.84, 0.15, 1.21, 0.87, 4.31, 1.48, 4.67, 2.35, 3.53, 4.42, 0.58, 2.04],
    [0.71, 4.33, 3.59, 2.86, 4.19, 0.32, 3.47, 0.69, 1.78, 2.65, 4.84, 1.51, 0.83],
    [1.69, 2.76, 4.84, 0.28, 4.66, 3.48, 1.67, 2.94, 0.39, 4.33, 3.56, 2.11, 1.28],
    [4.82, 0.12, 3.03, 4.44, 1.15, 0.67, 2.89, 4.15, 1.02, 3.46, 0.55, 4.91, 2.77],
    [3.01, 2.28, 1.16, 2.65, 0.73, 4.97, 3.88, 1.17, 4.66, 0.39, 4.41, 1.26, 3.35],
    [4.41, 3.94, 0.49, 1.86, 4.13, 0.68, 2.74, 3.18, 0.95, 4.57, 2.09, 1.42, 4.64],
    [0.54, 2.15, 4.70, 3.07, 1.79, 2.36, 4.84, 0.56, 2.25, 4.43, 1.67, 3.91, 0.88],
    [3.38, 1.07, 2.96, 4.51, 0.93, 3.26, 1.14, 4.73, 2.78, 1.29, 0.48, 4.38, 3.16],
    [2.45, 4.56, 1.64, 0.74, 3.48, 1.85, 0.69, 2.91, 4.37, 1.58, 3.04, 0.14, 4.92],
    [1.56, 0.38, 4.45, 3.67, 2.95, 4.82, 0.41, 3.28, 4.09, 1.47, 2.53, 4.63, 0.99],
    [3.67, 2.44, 1.23, 0.25, 4.96, 2.72, 1.85, 0.91, 4.52, 2.68, 0.43, 3.17, 4.09],
    [4.99, 3.11, 0.68, 2.81, 0.14, 4.31, 1.06, 2.59, 3.91, 0.75, 4.86, 2.04, 1.73],
    [0.82, 4.78, 3.39, 1.14, 2.67, 4.07, 3.25, 1.93, 0.55, 4.48, 3.13, 0.62, 2.85],
    [2.14, 1.83, 4.05, 0.42, 3.91, 0.95, 4.64, 2.46, 1.38, 4.29, 0.87, 3.88, 1.14],
    [4.66, 0.96, 3.70, 2.53, 4.23, 0.14, 3.04, 4.71, 1.69, 2.48, 0.38, 4.92, 3.66],
    [1.23, 3.17, 2.59, 0.79, 4.76, 3.27, 1.93, 0.55, 4.35, 3.08, 2.14, 4.46, 0.91],
    [3.84, 2.73, 0.51, 4.49, 0.87, 3.05, 0.65, 4.91, 1.23, 3.54, 0.42, 4.27, 2.36],
    [4.18, 1.42, 3.26, 0.68, 2.49, 4.88, 1.76, 3.14, 0.94, 4.62, 2.58, 1.49, 3.71],
    [0.37, 4.59, 2.07, 3.19, 0.82, 4.13, 2.54, 0.29, 4.65, 1.06, 3.72, 0.97, 4.41],
    [1.92, 0.45, 4.81, 2.35, 0.14, 4.75, 3.66, 1.54, 4.09, 2.81, 0.87, 3.93, 1.45],
    [3.73, 2.54, 1.34, 4.18, 1.92, 0.39, 4.87, 2.73, 0.95, 4.44, 3.12, 1.29, 4.52],
    [4.39, 0.91, 2.25, 0.48, 4.96, 3.05, 1.83, 0.26, 4.75, 3.91, 0.64, 2.38, 4.18],
    [2.05, 3.65, 0.14, 1.28, 4.13, 2.97, 0.37, 3.42, 4.59, 1.16, 2.71, 4.08, 0.84],
    [1.47, 4.82, 3.93, 0.59, 4.23, 1.76, 3.54, 0.83, 4.94, 2.68, 1.32, 4.39, 3.05]

]
}
"""


In [106]:
validate_serving_input(model_uri,serving_payload)

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

load the model back for prediction as a generic python function model

In [113]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions = loaded_model.predict(X_test)

heart_disease_prediction = dataset.drop(columns=["target"]).columns 

result = pd.DataFrame(X_test,columns=heart_disease_prediction)
result["actual_class"] = y_test
result["predicted_class"] = predictions

In [114]:
result

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,actual_class,predicted_class
0,57.0,1.0,0.0,150.0,276.0,0.0,0.0,112.0,1.0,0.6,1.0,1.0,1.0,0,0
1,59.0,1.0,3.0,170.0,288.0,0.0,0.0,159.0,0.0,0.2,1.0,0.0,3.0,0,1
2,57.0,1.0,2.0,150.0,126.0,1.0,1.0,173.0,0.0,0.2,2.0,1.0,3.0,1,1
3,56.0,0.0,0.0,134.0,409.0,0.0,0.0,150.0,1.0,1.9,1.0,2.0,3.0,0,0
4,71.0,0.0,2.0,110.0,265.0,1.0,0.0,130.0,0.0,0.0,2.0,1.0,2.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,69.0,1.0,2.0,140.0,254.0,0.0,0.0,146.0,0.0,2.0,1.0,3.0,3.0,0,0
57,50.0,1.0,2.0,129.0,196.0,0.0,1.0,163.0,0.0,0.0,2.0,0.0,2.0,1,1
58,68.0,1.0,0.0,144.0,193.0,1.0,1.0,141.0,0.0,3.4,1.0,2.0,3.0,0,0
59,60.0,1.0,0.0,145.0,282.0,0.0,0.0,142.0,1.0,2.8,1.0,2.0,3.0,0,0


inferencing from model from model registry

In [115]:
import mlflow.sklearn
model_name = "tracking-quickstart"
model_version = "latest"
model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri=model_uri)

In [116]:
model

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,88
,solver,'lbfgs'
,max_iter,1000
,multi_class,'ovr'


In [117]:
model_uri

'models:/tracking-quickstart/latest'

In [119]:
y_pred_new = model.predict(X_test)

In [120]:
y_pred_new

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0])